In [1]:
import torch

torch.manual_seed(0)
import random

random.seed(0)
import numpy as np

np.random.seed(0)

from omegaconf import OmegaConf
from secret_sauce.config.config import Config
from secret_sauce.dataset.datasources import DiskDataSource
from secret_sauce.dataset.songs_dataset import SongDataset
from secret_sauce.network.vqvae.vqvae import VQVAE
from secret_sauce.util.util import is_master, wait_for_debugger
import itertools

from performer_pytorch import PerformerLM
from performer_pytorch.autoregressive_wrapper import AutoregressiveWrapper
import torchaudio     
from torch.utils.data import DataLoader
from tqdm import tqdm

/root/secret_sauce/.venv/lib/python3.9/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
cfg = Config()
torch.set_autocast_enabled(True)

vqvae = VQVAE(cfg)
vq_save = torch.load('/root/secret_sauce/weights/04/epoch-29/mp_rank_00_model_states.pt')['module']
vqvae.load_state_dict(vq_save)
vqvae.to(device='cpu', dtype=torch.float32)
vqvae.eval()


VQVAE(
  (encoder): Sequential(
    (0): Conv1d(1, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): Sequential(
      (0): Resnet1dBlock(
        (model): Sequential(
          (0): ReLU()
          (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
          (2): ReLU()
          (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
        )
      )
      (1): Resnet1dBlock(
        (model): Sequential(
          (0): ReLU()
          (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(6,))
          (2): ReLU()
          (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
        )
      )
      (2): Resnet1dBlock(
        (model): Sequential(
          (0): ReLU()
          (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(9,), dilation=(9,))
          (2): ReLU()
          (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
        )
      )
      (3): Resnet1dBlock(
        (model): Sequential(
          (0): ReLU(

In [3]:
s = torch.load('/root/secret_sauce/sample.pt')
s.shape


torch.Size([12288])

In [11]:
t: torch.Tensor = s.to(device='cpu', dtype=torch.long).unsqueeze(0)
t.shape
t.

tensor([   3, 1545,  331,  ..., 1562,  417, 1562])

In [10]:
audio = vqvae.decode(t[:,:4096*1.5])[0]

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
audio.shape

In [ ]:
save_audio = audio[0].to(device='cpu', dtype=torch.float32)


In [ ]:
torchaudio.save('./song.mp3', save_audio, sample_rate=32000)